## Sort Oddity Files into Session Folders (.ps1)

VERSION: `Sort_Oddity_Converted_Raw.ps1`

### Advanced Explanation

This PowerShell script sorts `.edat2` and `.txt` files into folders based on a grouping criterion. It begins by fetching files with these extensions in the current directory (`Get-ChildItem -Path .\* -Include *.edat2, *.txt -File`). The files are grouped using `Group-Object`. If the base name matches a pattern (`'^[a-zA-Z]*(\d+)_`'), it groups by the numeric part; otherwise, it uses the entire base name.

The script iterates over each group. It first checks for the presence of an `.edat2` file in each group, skipping the group if none is found. The folder name is determined from the base name of the `.edat2` file. The script then checks for the presence of both `.edat2` and `.txt` files within the group. If both are present, it creates a new directory (if it doesn't exist) and moves all files in the group to this directory, updating counters for files and folders sorted.

If either file type is missing in a group, the missing file names are added to `$missingFiles`. After processing all groups, it lists any missing files and outputs the total count of sorted files and created folders.

### User-Friendly Explanation

This script helps to organize `.edat2` and `.txt` files into separate folders. It goes through all such files in a folder, and groups them based on their names. Each group is intended to have one `.edat2` file and one `.txt` file. It then creates a new folder for each group and moves the files into these folders.

The script also checks if any group is missing either a `.txt` or `.edat2` file. If a file is missing, it notes down which one is absent. After it finishes organizing, the script reports any missing files, how many files it moved, and how many new folders it created. This is useful for keeping your files neatly sorted and identifying any missing files in a set.

### Oversimplified Explantion

This should be saved in the processing folder of the testing laptop. It creates a new oddity session folder for every group of Oddity files in the current directory (2 txt and 1 edat2). 

In [ ]:
$files = Get-ChildItem -Path .\* -Include *.edat2, *.txt -File

$groupedFiles = $files | Group-Object { if ($_.BaseName -match '^[a-zA-Z]*(\d+)_') { $matches[1] } else { $_.BaseName } }

$sortedFileCount = 0
$sortedFolderCount = 0
$missingFiles = @()

foreach ($group in $groupedFiles) {
    $edat2File = $group.Group | Where-Object { $_.Extension -eq '.edat2' } | Select-Object -First 1
    if ($edat2File -eq $null) {
        continue 
    }
    
    $folderName = $edat2File.BaseName
    $directoryPath = Join-Path -Path (Get-Location) -ChildPath $folderName

    $hasTxt = $false
    $hasEdat2 = $false

    foreach ($file in $group.Group) {
        if ($file.Extension -eq '.txt') {
            $hasTxt = $true
        } elseif ($file.Extension -eq '.edat2') {
            $hasEdat2 = $true
        }
    }

    if ($hasTxt -and $hasEdat2) {
        if (-not (Test-Path $directoryPath)) {
            New-Item -Path $directoryPath -ItemType Directory
        }

        foreach ($file in $group.Group) {
            $newPath = Join-Path -Path $directoryPath -ChildPath $file.Name
            Move-Item -Path $file.FullName -Destination $newPath
            $sortedFileCount++
        }
        $sortedFolderCount++
    } else {
        if (-not $hasTxt) { $missingFiles += "$folderName.txt" }
        if (-not $hasEdat2) { $missingFiles += "$folderName.edat2" }
    }
}

if ($missingFiles.Count -gt 0) {
    Write-Output "Missing Files:"
    $missingFiles | ForEach-Object { Write-Output $_ }
}

Write-Output "Total Files Sorted: $sortedFileCount"
Write-Output "Total Folders Created: $sortedFolderCount"


VERSION: `Sort_Oddity_edat2&txt.ps1`

Expects edat2 and a raw txt 

In [ ]:
$files = Get-ChildItem -Path .\* -Include *.edat2, *.txt -File
$groupedFiles = $files | Group-Object { $_.BaseName }

$sortedFileCount = 0
$sortedFolderCount = 0
$missingFiles = @()

foreach ($group in $groupedFiles) {
    $folderName = $group.Name
    $directoryPath = Join-Path -Path (Get-Location) -ChildPath $folderName

    $hasTxt = $false
    $hasEdat2 = $false

    foreach ($file in $group.Group) {
        if ($file.Extension -eq '.txt') {
            $hasTxt = $true
        } elseif ($file.Extension -eq '.edat2') {
            $hasEdat2 = $true
        }
    }

    if ($hasTxt -and $hasEdat2) {
        if (-not (Test-Path $directoryPath)) {
            New-Item -Path $directoryPath -ItemType Directory
        }

        foreach ($file in $group.Group) {
            $newPath = Join-Path -Path $directoryPath -ChildPath $file.Name
            Move-Item -Path $file.FullName -Destination $newPath
            $sortedFileCount++
        }
        $sortedFolderCount++
    } else {
        if (-not $hasTxt) { $missingFiles += "$folderName.txt" }
        if (-not $hasEdat2) { $missingFiles += "$folderName.edat2" }
    }
}

if ($missingFiles.Count -gt 0) {
    Write-Output "Missing Files:"
    $missingFiles | ForEach-Object { Write-Output $_ }
}

Write-Output "Total Files Sorted: $sortedFileCount"
Write-Output "Total Folders Created: $sortedFolderCount"